# reader specifications

> Defines the generic classes to handle import of data (images & ROIs) to *findmycells* that will by extended by specific reader submodules (findmycells.readers.specs)

- order: 13

In [ ]:
#| default_exp readers/specs

In [ ]:
#| export

from abc import ABC, abstractmethod
from typing import List, Tuple, Optional, Dict, Any
from pathlib import PosixPath, Path, WindowsPath

from findmycells.configs import DefaultConfigs, GUIConfigs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

class ReaderSpecsABC(ABC):
    
    @property
    @abstractmethod
    def reader_type(self) -> str:
        pass
    
    @property
    @abstractmethod
    def reader_widget_description(self) -> str:
        """
        HTML syntax supported, as it will be used as "value" when constructing 
        an ipytwidgets.HTML object.
        """
        pass
    
    @property
    @abstractmethod
    def default_configs(self) -> DefaultConfigs:
        """
        Abstract method that requires its subclasses to define the `default_configs`
        as a property of the class. Thus, this will specify all configuration options
        that come with each subclass, while simultaneously also providing default values
        for each option and, moreover, defining what types of values are allowed for each
        option. Check out the implementation of `DefaultConfigs` in the configs module, or
        have a look at how this is implemented in one of the following ReaderSpecs 
        subclasses, like the `MicroscopyReaderSpecs` class.
        """
        pass    
    
    
    @property
    @abstractmethod
    def widget_names(self) -> Dict[str, str]:
        pass
    
    
    @property
    @abstractmethod
    def descriptions(self) -> Dict[str, str]:
        pass
    
    
    @property
    @abstractmethod
    def tooltips(self) -> Optional[Dict[str, str]]:
        return None    
    
    
    def initialize_gui_configs_and_widget(self) -> None:
        gui_configs = GUIConfigs(widget_names = self.widget_names,
                                 descriptions = self.descriptions,
                                 tooltips = self.tooltips)
        gui_configs.construct_widget(info_text = self.reader_widget_description,
                                     default_configs = self.default_configs)
        setattr(self, 'gui_configs', gui_configs)
        self.widget = self.gui_configs.strategy_widget
    
    
    def export_current_gui_config_values(self) -> Dict:
        return self.gui_configs.export_current_config_values()    

In [ ]:
#| export

class MicroscopyReaderSpecs(ReaderSpecsABC):
    
    @property
    def reader_type(self) -> str:
        return 'microscopy_images'
    
    @property
    def reader_widget_description(self) -> str:
        description = """
        <br>
        <div style="font-size: 13"> <b>User info:</b></div>
        <br>
        The following widgets help you in loading your microscopy image data into findmycells, 
        while also aiming at limiting the required computing and memory resources. Therfore, 
        you have the choice to select precisely those color channels or plane indices, which 
        shall be analyzed.
        <br>
        Not sure in which color channel index you can find your color of interest? Or you 
        want to try different options? Simply specify any combination that seems good to you 
        and see how your images will be loaded by heading over to the "preprocessing" step. 
        Feel free to use your favorite combination of preprocessing methods, or just go for 
        a plain conversion into 8-bit & then check the preprocessed images in the 
        "preprocessed_images" directory, which was created in the root directory you specified.
        If something does not look quite right just yet, simply come back here and re-iterate 
        this process until you're happy with the result.
        """
        return description    
    
    @property
    def widget_names(self):
        widget_names = {'all_color_channels': 'Checkbox',
                        'specific_color_channel_idxs_range': 'IntRangeSlider',
                        'all_planes': 'Checkbox',
                        'specific_plane_idxs_range': 'IntRangeSlider',
                        'version_idx': 'BoundedIntText',
                        'tile_row_idx': 'BoundedIntText',
                        'tile_col_idx': 'BoundedIntText'}
        return widget_names

    @property
    def descriptions(self):
        descriptions = {'all_color_channels': 'load all color channels (if unchecked, you have to specify the range color channel below)',
                        'specific_color_channel_idxs_range': 'Range of color channel indices to load (only if "all color channels" is unchecked)',
                        'all_planes': 'load all image planes (if unchecked, you have to specify the range of planes below)',
                        'specific_plane_idxs_range': 'Range of plane indices to load (only if "all planes" is unchecked)',
                        'version_idx': '[Optional] Version index to load',
                        'tile_row_idx': '[Optional] Row index of tile to load',
                        'tile_col_idx': '[Optional] Column index of tile to load'}
        return descriptions
    
    @property
    def tooltips(self):
        return {}   
    
    
    @property
    def default_configs(self) -> DefaultConfigs:
        default_values = {'all_color_channels': True,
                          'specific_color_channel_idxs_range': (0, 1),
                          'all_planes': True,
                          'specific_plane_idxs_range': (0, 1),
                          'version_idx': 0,
                          'tile_row_idx': 0,
                          'tile_col_idx': 0}
        
        valid_types = {'all_color_channels': [bool],
                       'specific_color_channel_idxs_range': [tuple], # a tuple of integers
                       'all_planes': [bool],
                       'specific_plane_idxs_range': [tuple], # a tuple of integers
                       'version_idx': [int],
                       'tile_row_idx': [int],
                       'tile_col_idx': [int]}
        
        valid_ranges = {'specific_color_channel_idxs_range': (0, 3, 1), # are more possible? most implementations currently assume single color or RGB
                        'specific_plane_idxs_range': (0, 100, 1), # more should usually not be required?
                        'version_idx': (0, 999, 1), # 999 just to put an upper limit
                        'tile_row_idx': (0, 999, 1), # 999 just to put an upper limit
                        'tile_col_idx': (0, 999, 1)} # 999 just to put an upper limit        
        default_configs = DefaultConfigs(default_values = default_values, valid_types = valid_types, valid_value_ranges = valid_ranges)
        return default_configs

In [ ]:
#| export

class ROIReaderSpecs(ReaderSpecsABC):
    
    """
    findmycells enables analyses of multiple ROIs in the image data. To do so, they will be matched based on their ID that will
    be retrieved from the ROI file. Some softwares that create these ROI-files, however, create default IDs for the individual
    ROIs that will interfere with this matching. For instance, in Fiji / ImageJ2, created ROIs get its centroid (?) pixel 
    coordinates as default ID (e.g. something like "523-378"). Since such default IDs most likely won´t be consistent throughout
    your entire image dataset, findmycells provides you with two options to adress this:
    a) You can set 'load_roi_ids_from_file' to False (default): 
      This will cause findmycells to ignore the IDs of the ROIs that are saved in the provided ROI file and assign them with 
      new IDs starting at "000". Note: Essentially, this requires you to have always the same type of ROIs present in the 
      exact same order in all your ROI-files. It is therefore only recommended if you have just a single ROI you´d like to analyze.
    b) You can set 'load_roi_ids_from_file' to True (recommended if you have more than a single ROI):
      This will enforce that findmycells uses the IDs that each ROI was saved with. Therefore, it requires that you use consistent
      naming of the ROIs with your preferred software. For instance, if you´re using Fiji / ImageJ2, you can rename each ROI in the
      ROIManager (e.g. "CA3", "vlPAG", or "ipsilateral_SNc"). Analyses and quantifications will then be matched and pooled across
      all ROIs with the respective IDs (e.g. all "CA3" ROIs).
    """
    
    @property
    def reader_type(self) -> str:
        return 'rois'
    
    @property
    def reader_widget_description(self) -> str:
        description = """
        <br>
        <div style="font-size: 13"> <b>User info:</b></div>
        <br>
        More often than not, you may be interested to quantify your specific image feature of interest 
        only in a very specific area of your image (or, <b>R</b>egion <b>O</b>f <b>I</b>nterest - ROI). 
        <i>findmycells</i> supports both: the quantification of image features in the entire image, 
        or precisely in one - or even multiple - specified ROI(s).<br>
        Interested in running whole image analysis? Well, simply make sure that "quantify whole image" 
        is checked, and your good to go!<br>
        Instead want to quantify the features in specifc ROIs? Not much harder! Please follow our examples 
        of how to create ROI-files, for instance using ImageJ2 (= Fiji). In case you want to quantify 
        multiple ROIs, please make sure to also check "load ROI IDs from file". <i>findmycells</i> will 
        then use this information and pool quantifaction across ROIs with the (exact!) same ID. Note: 
        This also applies if you have multiple different ROIs in your project, even if they are not 
        present in the same image.
        """
        return description
    
    @property
    def widget_names(self):
        widget_names = {'create_rois': 'Checkbox',
                        'load_roi_ids_from_file': 'Checkbox'}
        return widget_names

    @property
    def descriptions(self):
        descriptions = {'create_rois': 'Don`t use ROI files, and analyze whole images instead',
                        'load_roi_ids_from_file': 'load (= checked) or ingore (= unchecked) ROI IDs from ROI file or image name'}
        return descriptions
    
    @property
    def tooltips(self):
        return {}   
    
    
    @property
    def default_configs(self) -> DefaultConfigs:
        default_values = {'create_rois': False,
                          'load_roi_ids_from_file': True}
        valid_types = {'create_rois': [bool],
                       'load_roi_ids_from_file': [bool]}     
        default_configs = DefaultConfigs(default_values = default_values, valid_types = valid_types)
        return default_configs

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()